# model TPH

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import re
from dateutil.parser import parse
import string
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_percentage_error
import missingno as msno
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import scipy
import researchpy as rp
from matplotlib.offsetbox import AnchoredText
#import pacmap
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split 
# Import Halving Grid Search
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from scipy.stats import kurtosis,skew
from numpy import mean,sqrt,square
from scipy.fftpack import fft, fftfreq
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import PredefinedSplit
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [3]:
# Función que filtra tags por alta correlación
def corrkill(dataframe, features, corr_cut=0.8):

    df = dataframe[features]
    dfcorr = pd.DataFrame(np.triu(df.corr()), columns=df.columns, index=df.columns)
    dfcorr = dfcorr.stack().reset_index()
    dfcorr.columns = ['Feat1','Feat2','Val']
    dfcorr = dfcorr[~dfcorr['Val'].isin([0,1])]
    dfcorr = dfcorr[dfcorr['Val'].abs()>corr_cut]
    dfcorr["Val"]=dfcorr["Val"].abs()
    
    return list(dfcorr['Feat2']), dfcorr.sort_values(by="Val",ascending=False)

In [4]:
# read df full csv
df_features0=pd.read_csv('../../../data/processed data/df_feature_engineering_18December2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')#["2020":"2021"]
df_features0["TPH"]=df_features0["TPH"].shift(-10)
df_features0["loss of TPH"]=df_features0["loss of TPH"].shift(-10)
df_features0.dropna(inplace=True)
df_features0.sort_index(inplace=True)
df_features0.shape

(859325, 403)

In [9]:
# Verificando si hay duplicados
u=[i for i in df_features0.columns if (not re.match(".*water",i)) and (not re.match(".*power",i)) and (not re.match(".*delta HH TPH",i)) ]
y=[i for i in df_features0.columns if  re.match(".*sag power index",i)] 
u=y+u
df_features=df_features0[u].copy()
df_features.head(1)

,sag power index,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,sag power index_(t-1),sag power index_(t-2),sag power index_(t-3),sag power index_(t-4),sag power index_(t-5),pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,ball work index,bornite law,charge cell,TPH,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge 

# Equilibrio de eventos de perdidas de TPH

In [10]:
# Entrenamiento
entrenamiento=pd.concat([df_features["2020-01":"2020-03"],df_features["2020-05"],df_features["2020-07":"2020-12"],df_features["2021-01":"2021-08"],df_features["2021-10"],df_features["2021-12"]])
entrenamiento.head(1)

,sag power index,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,sag power index_(t-1),sag power index_(t-2),sag power index_(t-3),sag power index_(t-4),sag power index_(t-5),pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,ball work index,bornite law,charge cell,TPH,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge 

In [11]:
# Validación
validacion=pd.concat([df_features["2020-06"],df_features["2021-11"]])
validacion.head(1)

,sag power index,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,sag power index_(t-1),sag power index_(t-2),sag power index_(t-3),sag power index_(t-4),sag power index_(t-5),pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,ball work index,bornite law,charge cell,TPH,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge 

In [12]:
# Test
test=pd.concat([df_features["2020-04"],df_features["2021-09"],df_features["2022"]])
test.head(1)

,sag power index,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,sag power index_(t-1),sag power index_(t-2),sag power index_(t-3),sag power index_(t-4),sag power index_(t-5),pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,ball work index,bornite law,charge cell,TPH,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge 

In [13]:
tags=df_features.columns.to_list()
tags.remove("loss of TPH")
#tags.remove("TPH")

In [14]:
# Sobremuestreo: Aumentar numero de datos de la clase minoritaria
# ROS(Duplica clases):
ros=RandomOverSampler(random_state=0) # Random_state=0

# ROS:
Xtrain, Ytrain=ros.fit_resample(entrenamiento[tags],entrenamiento["loss of TPH"])

In [15]:
# Ytrain reg
#Ytrain=Xtrain["loss of TPH"]
Xtrain=Xtrain.drop(columns=["TPH"])

In [16]:
# Yval reg
Yval=validacion["loss of TPH"]
Xval=validacion.drop(columns=["loss of TPH","TPH"])

In [17]:
# Ytest reg
Ytest=test["loss of TPH"]
Xtest=test.drop(columns=["loss of TPH","TPH"])

In [18]:
#Unión del conjunto de Entrenamiento y Validación:
Entre_Vali_features=pd.concat([Xval,Xtrain])
Entre_Vali_target=pd.concat([Yval,Ytrain])

#Indices para separar ambos conjuntos:
Entre_indice=np.full(len(validacion),-1)
Vali_indice=np.full(len(entrenamiento),0)
Indices=np.append(Entre_indice,Vali_indice)
x=PredefinedSplit(Indices)

In [19]:
Color_loss=test["loss of TPH"]
Color_loss=Color_loss.astype(str)

# Feature selection

In [20]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        Xtrain.columns.to_list()
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("clf", xgb.XGBClassifier(seed=1,  eval_metric='mlogloss'))
    ]
)

In [21]:
pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                                  ['sag power index',
                                                   'mean_sag power index_10',
                                                   'max_sag power index_10',
                                                   'min_sag power index_10',
                                                   'rms_sag power index_10',
                                                   'var_sag power index_10',
                                                   'mean_sag power index_5',
                                                   'max_sag power index_5',
                                                   'min_sag power index_5',
                                                   'rms_sag power index_5',
                                                   'var_sag power index_5',
                                                  

**General**

In [22]:
# Métricas de evaluación 
Ypred= pipe.predict(Xtest)
print(classification_report(Ytest,Ypred))

              precision    recall  f1-score   support

         0.0       0.94      0.81      0.87     99821
         1.0       0.60      0.84      0.70     34164

    accuracy                           0.82    133985
   macro avg       0.77      0.83      0.79    133985
weighted avg       0.85      0.82      0.83    133985



In [23]:
df_importance=pd.DataFrame({"tag":Xtrain.columns,"importance":pipe["clf"].feature_importances_}).sort_values(by="importance",ascending=False).reset_index(drop=True)
df_importance.head()

,tag,importance
0,min_solid percentage_5,0.103160
1,solid percentage,0.101066
2,rms_speed_3,0.053070
3,rms_delta HH charge cell_3,0.044416
4,rms_delta LL charge cell_3,0.032669


In [24]:
pd.options.plotting.backend = "plotly"
fig=df_importance["importance"].plot(kind='hist')
fig.update_layout(height=500, width=1200,title="Feature importance XGBoost (Gain)",) 
fig.show()

In [25]:
# Plot
fig = go.Figure([go.Bar(x=df_importance["tag"], y=df_importance["importance"])])
fig.update_layout(height=500, width=1200,title="Feature importance XGBoost (Gain)",) 
fig.show()

In [26]:
xgboost_fs=df_importance[:100].tag.to_list()
xgboost_fs

['min_solid percentage_5',
 'solid percentage',
 'rms_speed_3',
 'rms_delta HH charge cell_3',
 'rms_delta LL charge cell_3',
 'var_delta LL charge cell_3',
 'min_solid percentage_3',
 'min_delta LL charge cell_10',
 'var_delta LL charge cell_10',
 'max_delta LL charge cell_10',
 'HH charge cell_(t-1)',
 'var_delta HH charge cell_10',
 'mean_LL charge cell_3',
 'rms_HH charge cell_10',
 'min_speed_10',
 'min_speed_3',
 'rms_delta LL charge cell_10',
 'max_LL charge cell_5',
 'chalcocite law_(t-2)',
 'min_speed_5',
 'delta LL charge cell',
 'mean_HH charge cell_3',
 'var_solid percentage_5',
 'max_sag power index_3',
 'mean_HH charge cell_5',
 'rms_sag power index_10',
 'min_LL charge cell_3',
 'min_chalcocite law_5',
 'max_HH charge cell_5',
 'bornite law_(t-4)',
 'min_solid percentage_10',
 'min_LL charge cell_10',
 'rms_chalcocite law_5',
 'LL charge cell',
 'delta HH charge cell',
 'min_pyrite law_3',
 'rms_bornite law_3',
 'bornite law_(t-3)',
 'max_chalcocite law_10',
 'var_solid 

In [27]:
# Se obtienen tags altamente correlacionados y matriz que permite visualizar cuales son los par de tags altamente correlacionados
collinearity, table_cor=corrkill(Xtrain,xgboost_fs,corr_cut=0.9)

In [28]:
# Tags altamente correlacionados
table_cor

,Feat1,Feat2,Val
1221,mean_LL charge cell_3,mean_HH charge cell_3,0.999996
1728,max_LL charge cell_5,max_HH charge cell_5,0.999996
3394,LL charge cell,HH charge cell,0.999996
8995,max_LL charge cell_10,max_HH charge cell_10,0.999996
1399,rms_HH charge cell_10,mean_LL charge cell_10,0.999996
...,...,...,...
934,max_delta LL charge cell_10,delta HH charge cell,0.909989
364,rms_delta HH charge cell_3,max_delta HH charge cell_10,0.906963
307,rms_delta HH charge cell_3,min_delta LL charge cell_10,0.906956
4287,min_delta LL charge cell_5,min_delta HH charge cell_10,0.904008


In [29]:
tag_select = [i for i in xgboost_fs if i not in collinearity]
tag_select

['min_solid percentage_5',
 'rms_speed_3',
 'rms_delta HH charge cell_3',
 'rms_delta LL charge cell_3',
 'var_delta LL charge cell_3',
 'var_delta LL charge cell_10',
 'max_delta LL charge cell_10',
 'HH charge cell_(t-1)',
 'chalcocite law_(t-2)',
 'var_solid percentage_5',
 'max_sag power index_3',
 'bornite law_(t-4)',
 'min_pyrite law_3',
 'var_solid percentage_10',
 'var_speed_10',
 'rms_granulometry_5',
 'ball work index_(t-1)',
 'rms_crusher index_5',
 'HH TPH',
 'min_chalcopyrite law_3',
 'var_covelin law_3',
 'var_charge cell_3',
 'var_LL charge cell_10',
 'var_solid percentage_3',
 'max_covelin law_3',
 'Edad',
 'var_covelin law_10']

In [30]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        tag_select
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("clf", xgb.XGBClassifier(seed=1,  eval_metric='mlogloss'))
    ]
)

In [31]:
pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                                  ['min_solid percentage_5',
                                                   'rms_speed_3',
                                                   'rms_delta HH charge cell_3',
                                                   'rms_delta LL charge cell_3',
                                                   'var_delta LL charge cell_3',
                                                   'var_delta LL charge '
                                                   'cell_10',
                                                   'max_delta LL charge '
                                                   'cell_10',
                                                   'HH charge cell_(t-1)',
                                                   'chalcocite law_(t-2)',
                                                   'var_solid percentage_5',
  

**General**

In [32]:
# Métricas de evaluación 
Ypred= pipe.predict(Xtest)
print(classification_report(Ytest,Ypred))

              precision    recall  f1-score   support

         0.0       0.93      0.80      0.86     99821
         1.0       0.59      0.83      0.69     34164

    accuracy                           0.81    133985
   macro avg       0.76      0.81      0.77    133985
weighted avg       0.84      0.81      0.82    133985



In [33]:
Ypred=pd.Series([1 if np.random.rand()<0.5 else 0 for i in range(len(Ytest))])
print(classification_report( Ytest,Ypred))

              precision    recall  f1-score   support

         0.0       0.75      0.50      0.60     99821
         1.0       0.26      0.50      0.34     34164

    accuracy                           0.50    133985
   macro avg       0.50      0.50      0.47    133985
weighted avg       0.62      0.50      0.53    133985



# GridSearch

In [34]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        tag_select
        #features.columns.to_list()
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
        #("Selection", SelectPercentile(f_classif, percentile=10)),
        #("PCA",PCA(n_components=150 )), #0.08
     ("reg",  xgb.XGBClassifier(seed=1,  eval_metric='mlogloss'))
    ]
)


param_grid_reg = {'reg__learning_rate': list(np.linspace(0.001,1,10)), # Boosting learning rate
                  'reg__gamma':list(np.linspace(0.0001,50,5)), # Minimum loss reduction required to make a further partition on a leaf node of the tree.
                  #'reg__max_depth':[5,10,12,15,20], # Maximum tree depth for base learners.
                  #'reg__min_child_weight':[10,5,1,2,3], # Minimum sum of instance weight(hessian) needed in a child.
                  #'reg__colsample_bytree':[0.5,0.8,1], # Subsample ratio of columns when constructing each tree.
                  'reg__n_estimators': [100,200,300], # Number of gradient boosted trees.   
                  #'reg__reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
              #'reg__reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
               #'Selection__percentile': [10,20,30,50,70,80,100]              
                 } 

# Clasificación
gs_reg = HalvingGridSearchCV(pipe, param_grid_reg, scoring='neg_mean_absolute_error',cv=x,refit=True,
                             verbose=10,
                             aggressive_elimination=True
                            )

# Fit
gs_reg.fit(Entre_Vali_features, Entre_Vali_target)


n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 12955
max_resources_: 1049417
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 150
n_resources: 12955
Fitting 1 folds for each of 150 candidates, totalling 150 fits
[CV 1/1; 1/150] START reg__gamma=0.0001, reg__learning_rate=0.001, reg__n_estimators=100
[CV 1/1; 1/150] END reg__gamma=0.0001, reg__learning_rate=0.001, reg__n_estimators=100;, score=(train=-0.078, test=-0.206) total time=   0.7s
[CV 1/1; 2/150] START reg__gamma=0.0001, reg__learning_rate=0.001, reg__n_estimators=200
[CV 1/1; 2/150] END reg__gamma=0.0001, reg__learning_rate=0.001, reg__n_estimators=200;, score=(train=-0.078, test=-0.196) total time=   0.4s
[CV 1/1; 3/150] START reg__gamma=0.0001, reg__learning_rate=0.001, reg__n_estimators=300
[CV 1/1; 3/150] END reg__gamma=0.0001, reg__learning_rate=0.001, reg__n_estimators=300;, score=(train=-0.074, test=-0.198) total time=   0.6s
[CV 1/1; 4/150] START reg__gamma=0.00

HalvingGridSearchCV(aggressive_elimination=True,
                    cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                    estimator=Pipeline(steps=[('preprocesamiento',
                                               ColumnTransformer(transformers=[('MinMax',
                                                                                MinMaxScaler(),
                                                                                ['min_solid '
                                                                                 'percentage_5',
                                                                                 'rms_speed_3',
                                                                                 'rms_delta '
                                                                                 'HH '
                                                                                 'charge '
                                                           

In [35]:
gs_reg.best_params_

{'reg__gamma': 25.000049999999998,
 'reg__learning_rate': 0.112,
 'reg__n_estimators': 300}

In [36]:
# Métricas de evaluación 
Ypred= gs_reg.predict(Xtest)
print(classification_report(Ytest,Ypred))

              precision    recall  f1-score   support

         0.0       0.93      0.82      0.87     99821
         1.0       0.61      0.83      0.70     34164

    accuracy                           0.82    133985
   macro avg       0.77      0.83      0.79    133985
weighted avg       0.85      0.82      0.83    133985



# Train model

In [37]:
tag_select

['min_solid percentage_5',
 'rms_speed_3',
 'rms_delta HH charge cell_3',
 'rms_delta LL charge cell_3',
 'var_delta LL charge cell_3',
 'var_delta LL charge cell_10',
 'max_delta LL charge cell_10',
 'HH charge cell_(t-1)',
 'chalcocite law_(t-2)',
 'var_solid percentage_5',
 'max_sag power index_3',
 'bornite law_(t-4)',
 'min_pyrite law_3',
 'var_solid percentage_10',
 'var_speed_10',
 'rms_granulometry_5',
 'ball work index_(t-1)',
 'rms_crusher index_5',
 'HH TPH',
 'min_chalcopyrite law_3',
 'var_covelin law_3',
 'var_charge cell_3',
 'var_LL charge cell_10',
 'var_solid percentage_3',
 'max_covelin law_3',
 'Edad',
 'var_covelin law_10']

In [38]:
tag_select=['min_solid percentage_5',
 'rms_speed_3',
 'rms_delta HH charge cell_3',
 'rms_delta LL charge cell_3',
 'var_delta LL charge cell_3',
 'var_delta LL charge cell_10',
 'max_delta LL charge cell_10',
 'HH charge cell_(t-1)',
 'chalcocite law_(t-2)',
 'var_solid percentage_5',
 'max_sag power index_3',
 'bornite law_(t-4)',
 'min_pyrite law_3',
 'var_solid percentage_10',
 'var_speed_10',
 'rms_granulometry_5',
 'ball work index_(t-1)',
 'rms_crusher index_5',
 'HH TPH',
 'min_chalcopyrite law_3',
 'var_covelin law_3',
 'var_charge cell_3',
 'var_LL charge cell_10',
 'var_solid percentage_3',
 'max_covelin law_3',
 'Edad',
 'var_covelin law_10']

In [39]:
gs_reg.best_params_

{'reg__gamma': 25.000049999999998,
 'reg__learning_rate': 0.112,
 'reg__n_estimators': 300}

In [40]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        tag_select
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("clf",   xgb.XGBClassifier(seed=1,  eval_metric='mlogloss',gamma= 25, learning_rate= 0.112, n_estimators= 300))
    ]
)

pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                                  ['min_solid percentage_5',
                                                   'rms_speed_3',
                                                   'rms_delta HH charge cell_3',
                                                   'rms_delta LL charge cell_3',
                                                   'var_delta LL charge cell_3',
                                                   'var_delta LL charge '
                                                   'cell_10',
                                                   'max_delta LL charge '
                                                   'cell_10',
                                                   'HH charge cell_(t-1)',
                                                   'chalcocite law_(t-2)',
                                                   'var_solid percentage_5',
  

**General**

In [41]:
# Métricas de evaluación 
Ypred= pipe.predict(Xtest)
print(classification_report(Ytest,Ypred))

              precision    recall  f1-score   support

         0.0       0.94      0.81      0.87     99821
         1.0       0.60      0.85      0.70     34164

    accuracy                           0.82    133985
   macro avg       0.77      0.83      0.79    133985
weighted avg       0.85      0.82      0.83    133985



In [42]:
# Matriz de confusión
matrix=confusion_matrix(Ytest,Ypred,normalize='true'
)
z=[matrix[0].round(2),matrix[1].round(2)]
fig = px.imshow(z, aspect="auto",labels=dict(x="Predicted label", y="True label"),x=["0", "1"],y=["0", "1"]
)

#axis_template = dict(showticklabels = False,ticks = '')
fig.update_layout(height=600,width=600,#title="Matriz de confusión normalizada",
# xaxis = axis_template,yaxis = axis_template,
)
fig.update_xaxes(side="top")
for i,r in enumerate(matrix):
    for k,c in enumerate(r):
        fig.add_annotation(x=k,y=i,
                           text='{}'.format(round(c,2)),
                           showarrow=False, bgcolor="white"
    )
                          
fig.show()

In [43]:
# Matriz de confusión
matrix=confusion_matrix(Ytest,Ypred,#normalize='true'
)
z=[matrix[0].round(2),matrix[1].round(2)]
fig = px.imshow(z, aspect="auto",labels=dict(x="Predicted label", y="True label"),x=["0", "1"],y=["0", "1"]
)

#axis_template = dict(showticklabels = False,ticks = '')
fig.update_layout(height=600,width=600,#title="Matriz de confusión normalizada",
# xaxis = axis_template,yaxis = axis_template,
)
fig.update_xaxes(side="top")
for i,r in enumerate(matrix):
    for k,c in enumerate(r):
        fig.add_annotation(x=k,y=i,
                           text='{}'.format(round(c,2)),
                           showarrow=False, bgcolor="white"
    )
                          
fig.show()